# Tensor (Basic)

In [1]:
import torch
import numpy as np

: 

```
TENSOR DIMENSIONS (Visual Guide):

0D (Scalar):  [5.2]
             (just a number)

1D (Vector):  [1.2, 3.4, 5.6, 7.8]
             (like pixel intensities along a line)

2D (Matrix):  [[120, 150, 180],     ← Row 0
               [90,  110, 140],     ← Row 1  
               [60,  80,  100]]     ← Row 2
              ↑     ↑     ↑
           Col 0  Col 1  Col 2
           (like a grayscale image)

3D Tensor:    Multiple 2D slices stacked
              (like CT scan slices)
              
4D Tensor:    [Batch, Channels, Height, Width]
              (multiple images for training)
```

## Create Tensor

### From Python lists

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [4]:
pixel_values = [[120, 150, 180], [90, 110, 140], [60, 80, 100]]
image_tensor = torch.tensor(pixel_values, dtype=torch.float32)
image_tensor


tensor([[120., 150., 180.],
        [ 90., 110., 140.],
        [ 60.,  80., 100.]])

In [5]:
image_tensor.shape

torch.Size([3, 3])

In [6]:
image_tensor.dtype

torch.float32

### From NumPy arrays

In [9]:
np_image = np.random.randint(0, 255, size=(64, 64), dtype=np.uint8)
np_image


array([[ 64, 175,  82, ...,  15,  81, 218],
       [251, 226, 178, ..., 219, 136, 232],
       [238, 226,  33, ..., 243,  55,  14],
       ...,
       [183,  77, 146, ..., 208,  33, 132],
       [208,  36, 247, ...,   3, 136,  44],
       [ 30, 202, 102, ..., 158,  51, 181]], shape=(64, 64), dtype=uint8)

In [12]:
medical_image = torch.from_numpy(np_image)
medical_image

tensor([[ 64., 175.,  82.,  ...,  15.,  81., 218.],
        [251., 226., 178.,  ..., 219., 136., 232.],
        [238., 226.,  33.,  ..., 243.,  55.,  14.],
        ...,
        [183.,  77., 146.,  ..., 208.,  33., 132.],
        [208.,  36., 247.,  ...,   3., 136.,  44.],
        [ 30., 202., 102.,  ..., 158.,  51., 181.]])

### From Tensor

In [15]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [16]:
# retains the properties of x_data
x_ones = torch.ones_like(x_data)
x_ones

tensor([[1, 1],
        [1, 1]])

In [18]:
# overrides the datatype of x_data
x_rand = torch.rand_like(x_data, dtype=torch.float) 
x_rand

tensor([[0.5339, 0.4105],
        [0.1034, 0.5003]])

In [25]:
# Built-in tensor creation functions
zeros_like_scan = torch.zeros(512, 512)  # Like an empty 512x512 image
ones_mask = torch.ones(256, 256)         # Like a binary mask
random_noise = torch.randn(100, 100)     # Gaussian noise (useful for testing)


## Attributes of a Tensor

Tensor attributes describe their shape, datatype, and the device on which they are stored.

In [19]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## Operations on Tensors

Each of these operations can be run on the CPU and Accelerator such as CUDA, MPS, MTIA, or XPU. 


### Accelerator

In [21]:
torch.accelerator.is_available()

True

In [23]:
torch.accelerator.current_accelerator() # MacOS

device(type='mps')

In [20]:
# We move our tensor to the current accelerator if available
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

## Radiology Example

In [29]:
# Create a 3D tensor representing a small CT volume (depth, height, width)
ct_volume = torch.randn(32, 128, 128)  # 32 slices, 128x128 pixels each

### Index and Slice

In [30]:
print(f"CT Volume shape: {ct_volume.shape}")
print(f"Number of dimensions: {ct_volume.ndim}")
print(f"Total number of elements: {ct_volume.numel()}")
print(f"Data type: {ct_volume.dtype}")
print(f"Device (CPU/GPU): {ct_volume.device}\n")

CT Volume shape: torch.Size([32, 128, 128])
Number of dimensions: 3
Total number of elements: 524288
Data type: torch.float32
Device (CPU/GPU): cpu



In [31]:
# Example Index & Slicing
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [32]:
# Indexing and slicing (like selecting ROI in medical images)
first_slice = ct_volume[0]           # Get first slice
roi_patch = ct_volume[10:15, 50:100, 50:100]  # 5 slices, specific region
center_pixel = ct_volume[16, 64, 64]  # Single voxel value

print(f"First slice shape: {first_slice.shape}")
print(f"ROI patch shape: {roi_patch.shape}")
print(f"Center pixel value: {center_pixel.item():.3f}\n")

First slice shape: torch.Size([128, 128])
ROI patch shape: torch.Size([5, 50, 50])
Center pixel value: 0.562



### Reshaping tensors

In [33]:
original_shape = ct_volume.shape
flattened = ct_volume.view(-1)  # Flatten to 1D (like vectorizing an image)
reshaped = ct_volume.view(32, -1)  # Keep first dim, flatten others

print(f"Original: {original_shape}")
print(f"Flattened: {flattened.shape}")
print(f"Reshaped: {reshaped.shape}\n")

Original: torch.Size([32, 128, 128])
Flattened: torch.Size([524288])
Reshaped: torch.Size([32, 16384])



### Element-wise Operation

In [34]:
image_a = torch.randn(256, 256)
image_b = torch.randn(256, 256)

# Element-wise operations (applied to each pixel)
sum_images = image_a + image_b       # Add two images
scaled_image = image_a * 0.5         # Scale intensity
normalized = (image_a - image_a.mean()) / image_a.std()  # Z-score normalization

print(f"Original image stats - mean: {image_a.mean():.3f}, std: {image_a.std():.3f}")
print(f"Normalized image stats - mean: {normalized.mean():.3f}, std: {normalized.std():.3f}\n")


Original image stats - mean: 0.004, std: 0.996
Normalized image stats - mean: -0.000, std: 1.000



### Arithmatic Operation

#### Matrix Multiplication

In [72]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

In [78]:
(y1 == y2) & (y2 == y3)

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

#### Element-wise Product

In [79]:
# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

#### Single Element Tensor

In [80]:
agg = tensor.sum()
agg

tensor(12.)

In [82]:
# you can convert it to a Python numerical value using item():
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


### In-place operations

Operations that store the result into the operand are called in-place. They are denoted by a `_` suffix.

In [83]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


## Other Tensor Manipulation

### Transpose operations

In [35]:
image_2d = torch.randn(480, 640)  # Height x Width
transposed = image_2d.t()         # Width x Height (transpose)
print(f"Original image: {image_2d.shape}")
print(f"Transposed: {transposed.shape}\n")

Original image: torch.Size([480, 640])
Transposed: torch.Size([640, 480])



### Permute Axis

In [ ]:
# Permute for more complex dimension reordering
batch_images = torch.randn(10, 3, 224, 224)  # Batch, Channels, Height, Width
# Change to Batch, Height, Width, Channels (for some visualization libraries)
reordered = batch_images.permute(0, 2, 3, 1)
print(f"Original batch: {batch_images.shape}")
print(f"Reordered: {reordered.shape}\n")

### Concatenation

In [38]:
image_2d.shape

torch.Size([480, 640])

In [36]:
# Cat for concatenating along existing dimension
extended_image = torch.cat([image_2d, image_2d], dim=1)  # Side by side
print(f"Extended image (side by side): {extended_image.shape}")


Extended image (side by side): torch.Size([480, 1280])


### Create Stack Volume

In [40]:
# Concatenation (joining tensors)
slice_1 = torch.randn(128, 128)
slice_2 = torch.randn(128, 128) 
slice_3 = torch.randn(128, 128)

In [41]:
# Stack to create a volume
volume = torch.stack([slice_1, slice_2, slice_3], dim=0)
print(f"Individual slice: {slice_1.shape}")
print(f"Stacked volume: {volume.shape}\n")

Individual slice: torch.Size([128, 128])
Stacked volume: torch.Size([3, 128, 128])



## Image Processing Example

In [43]:
# Simulate loading a batch of medical images for training
batch_size = 4
num_slices = 16
height, width = 256, 256

In [44]:
# Create a batch of 3D medical volumes
medical_batch = torch.randn(batch_size, num_slices, height, width)
print(f"Medical batch shape: {medical_batch.shape}")

Medical batch shape: torch.Size([4, 16, 256, 256])


### Normalize each volume independently

In [51]:
medical_batch[0].shape

torch.Size([16, 256, 256])

In [48]:
normalized_batch = torch.zeros_like(medical_batch)
for i in range(batch_size):
    volume = medical_batch[i]
    normalized_batch[i] = (volume - volume.mean()) / (volume.std() + 1e-8)

In [52]:
normalized_batch.shape

torch.Size([4, 16, 256, 256])

### Extract middle slices 

(common in 2D analysis)

In [ ]:

middle_slices = medical_batch[:, num_slices//2, :, :]  # Shape: (batch_size, height, width)
print(f"Middle slices shape: {middle_slices.shape}")

### Create a simple feature map (like edge detection)

In [ ]:
# This simulates what a convolutional layer might do
sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32)
